# Pump Locker Test - Dual HiScroll12 Control System

This notebook provides control and monitoring capabilities for two HiScroll12 vacuum pumps with TPG366 pressure sensors using the current hardware versions.

## Features:
- Control for 2 HiScroll12 vacuum pumps (current version)
- Monitoring for TPG366 pressure sensors (current version)
- Shared logging system using loguru
- Cancelable continuous monitoring
- Real-time data visualization
- Simplified and optimized for current hardware

## 1. Import Required Libraries and Setup

In [1]:
import sys
import threading
import time
from datetime import datetime
import asyncio
from typing import Dict, Optional, Any
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets
import pickle

from loguru import logger
import os
from pathlib import Path

# Add path to src modules
sys.path.append(os.path.join(os.getcwd(), '..', '..', 'src'))

# Import current device modules
from devices.pfeiffer.tpg366.tpg366 import TPG366

## 2. Setup Shared Logging System

In [2]:
# Get the repository root and create shared logs directory
repo_root = Path(os.getcwd()).parent.parent
log_dir = repo_root / "debugging" / "logs"
log_dir.mkdir(parents=True, exist_ok=True)

# Create shared log file for all devices
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
shared_log_file = log_dir / f"008_single_pressure_monitor_{timestamp}.log"

# Configure shared logger
logger.remove()  # Remove default logger

# Add console logger with INFO level
logger.add(sys.stderr, level="INFO", format="{time:YYYY-MM-DD HH:mm:ss} | {level} | {message}")

# Add shared file logger with DEBUG level
logger.add(
    str(shared_log_file),
    level="DEBUG",
    format="{time:YYYY-MM-DD HH:mm:ss.SSS} | {level} | {name}:{function}:{line} | {message}",
    rotation="1 day",
    retention="30 days",
    compression="zip"
)

logger.info("Single Pressure Monitor system initialized with shared logging")
print(f"Repository root: {repo_root}")
print(f"Shared logs will be saved to: {shared_log_file}")

2025-09-09 14:20:39 | INFO | Single Pressure Monitor system initialized with shared logging


Repository root: C:\Users\ESIBDlab\PycharmProjects\esibd_bs
Shared logs will be saved to: C:\Users\ESIBDlab\PycharmProjects\esibd_bs\debugging\logs\008_single_pressure_monitor_20250909_142039.log


## 3. Simplified Pump Controller Class

In [3]:
class PressureController:
    """Simplified controller class for managing two HiScroll12 pumps and TPG366 pressure sensors"""
    
    def __init__(self):
        self.pressure_controller: Optional[TPG366] = None
        
        # Monitoring control
        self.monitoring_active = False
        self.monitoring_thread = None
        self.monitoring_stop_event = threading.Event()
        
        # Data storage
        self.monitoring_data = []
        
        # Shared thread lock for all devices
        self.shared_lock = threading.Lock()
        
        logger.info("Pressure Controller initialized")
    
    def connect_pressure_controller(self, com_port: str, address: int):
        """Connect to TPG366 pressure controller with shared logger"""
        try:
            logger.info(f"Connecting to TPG366 pressure controller on {com_port}, address {address}")
            self.pressure_controller = TPG366(
                device_id="tpg366_pressure",
                port=com_port,
                device_address=address,
                logger=logger,  # Pass shared logger
                thread_lock=self.shared_lock  # Use shared lock
            )
            self.pressure_controller.connect()
            
            logger.success("TPG366 pressure controller connected successfully")
            return True
            
        except Exception as e:
            logger.error(f"Failed to connect TPG366 pressure controller: {e}")
            return False
    
    def disconnect_all(self):
        """Safely disconnect all devices"""
        logger.info("Disconnecting all devices...")
        
        # Stop monitoring first
        self.stop_monitoring()
        
        # Disconnect pressure controller
        if self.pressure_controller:
            try:
                self.pressure_controller.disconnect()
                logger.info("TPG366 pressure controller disconnected")
            except Exception as e:
                logger.error(f"Error disconnecting TPG366 pressure controller: {e}")
        
        logger.success("All devices disconnected")

# Create global controller instance
controller = PressureController()

2025-09-09 14:20:42 | INFO | Pressure Controller initialized


## 4. Device Monitoring Functions

In [4]:
def monitor_tpg366_pressure(pressure_controller: TPG366, sensor_id: str, channel: int) -> Dict[str, Any]:
    """
    Monitor pressure from TPG366 sensor channel.
    
    Parameters:
    - pressure_controller: TPG366 pressure controller instance
    - sensor_id: Identifier string for the sensor
    - channel: Sensor channel to read from (1-6)
    
    Returns:
    - Dictionary containing pressure data
    """
    
    try:
        logger.debug(f"Reading pressure from {sensor_id}, channel {channel}...")
        
        # Get actual pressure reading using current API
        pressure_value = pressure_controller.read_pressure_value(channel)
        
        pressure_data = {
            'timestamp': datetime.now(),
            'sensor_id': sensor_id,
            'channel': channel,
            'pressure_value': pressure_value,
            'status': 'OK'
        }
        
        logger.debug(f"{sensor_id} pressure reading: {pressure_value}")
        return pressure_data
        
    except Exception as e:
        logger.error(f"Error reading pressure from {sensor_id}: {e}")
        return {
            'timestamp': datetime.now(),
            'sensor_id': sensor_id,
            'channel': channel,
            'pressure_value': None,
            'error': str(e),
            'status': 'ERROR'
        }

print("Monitoring functions defined successfully")

Monitoring functions defined successfully


## 5. Continuous Monitoring System

In [5]:
def continuous_monitoring_worker(controller: PressureController, interval: float = 5.0):
    """
    Worker function for continuous monitoring that runs in a separate thread.
    """
    logger.info(f"Starting continuous monitoring with {interval}s interval")
    
    while not controller.monitoring_stop_event.is_set():
        try:
            monitoring_cycle_data = {
                'cycle_timestamp': datetime.now(),
                'sensor1_data': None
            }

            # Monitor pressure sensors if controller is connected
            if controller.pressure_controller:
                # Read from sensor channels 1 and 2
                monitoring_cycle_data['sensor1_data'] = monitor_tpg366_pressure(
                    controller.pressure_controller, "Sensor_1", channel=1
                )
            
            # Store data
            controller.monitoring_data.append(monitoring_cycle_data)
            
            # Log summary with pressure values if available
            log_msg = f"Monitoring cycle completed at {monitoring_cycle_data['cycle_timestamp']}"
            if monitoring_cycle_data['sensor1_data']:
                p1 = monitoring_cycle_data['sensor1_data'].get('pressure_value')
                log_msg += f" | Pressures: S1={p1}"
            logger.info(log_msg)
            
            # Wait for next cycle or stop signal
            if controller.monitoring_stop_event.wait(timeout=interval):
                break  # Stop event was set
                
        except Exception as e:
            logger.error(f"Error in monitoring cycle: {e}")
            # Continue monitoring even if one cycle fails
            time.sleep(interval)
    
    logger.info("Continuous monitoring stopped")

def start_monitoring(controller: PressureController, interval: float = 5.0):
    """Start continuous monitoring in a separate thread."""
    if controller.monitoring_active:
        logger.warning("Monitoring is already active")
        return False
    
    # Reset stop event and clear old data
    controller.monitoring_stop_event.clear()
    controller.monitoring_data.clear()
    
    # Start monitoring thread
    controller.monitoring_thread = threading.Thread(
        target=continuous_monitoring_worker,
        args=(controller, interval),
        daemon=True
    )
    
    controller.monitoring_thread.start()
    controller.monitoring_active = True
    
    logger.success(f"Continuous monitoring started with {interval}s interval")
    return True

def stop_monitoring(controller: PressureController):
    """Stop continuous monitoring gracefully."""
    if not controller.monitoring_active:
        logger.info("Monitoring is not currently active")
        return
    
    logger.info("Stopping continuous monitoring...")
    
    # Signal the monitoring thread to stop
    controller.monitoring_stop_event.set()
    
    # Wait for thread to finish (with timeout)
    if controller.monitoring_thread and controller.monitoring_thread.is_alive():
        controller.monitoring_thread.join(timeout=10.0)
        
        if controller.monitoring_thread.is_alive():
            logger.warning("Monitoring thread did not stop gracefully")
        else:
            logger.success("Monitoring thread stopped successfully")
    
    controller.monitoring_active = False
    controller.monitoring_thread = None

# Add methods to controller class
controller.start_monitoring = lambda interval=5.0: start_monitoring(controller, interval)
controller.stop_monitoring = lambda: stop_monitoring(controller)

print("Continuous monitoring system ready")

Continuous monitoring system ready


## 6. Device Connection Configuration

In [6]:
# Configuration for your devices
# Modify these values according to your setup

# TPG366 pressure controller
PRESSURE_COM = 'COM22'  # TPG366 pressure controller COM port
PRESSURE_ADDR = 10      # TPG366 standard address is 1

print("Device configuration loaded:")
print(f"TPG366 Pressure Controller: {PRESSURE_COM}, Address {PRESSURE_ADDR}")
print(f"  -> Sensor 1: Channel 1")

Device configuration loaded:
TPG366 Pressure Controller: COM22, Address 10
  -> Sensor 1: Channel 1


In [7]:
# Connect to TPG366 pressure controller
pressure_connection = controller.connect_pressure_controller(PRESSURE_COM, PRESSURE_ADDR)

if pressure_connection:
    print("✅ TPG366 pressure controller connected successfully")
    print("Sensor accessible via channels 1")
else:
    print("❌ Failed to connect to TPG366 pressure controller")

2025-09-09 14:20:56 | INFO | Connecting to TPG366 pressure controller on COM22, address 10
2025-09-09 14:20:56 | INFO | Connecting to Pfeiffer device tpg366_pressure on COM22
2025-09-09 14:20:56 | INFO | Successfully connected to device at address 10
2025-09-09 14:20:56 | SUCCESS | TPG366 pressure controller connected successfully


✅ TPG366 pressure controller connected successfully
Sensor accessible via channels 1


## 7. Device Testing and Status

In [14]:
controller.pressure_controller.read_pressure_value(1)

1006.9999999999999

In [15]:
# Test TPG366 pressure controller and sensors
if controller.pressure_controller:
    try:
        print("Testing TPG366 Pressure Controller:")
        
        # Get software version
        sw_version = controller.pressure_controller.get_software_version()
        print(f"  Software Version: {sw_version}")
        
        # Get device name
        device_name = controller.pressure_controller.get_electronics_name()
        print(f"  Device Name: {device_name}")
        
        # Test Sensor 1 (Channel 1)
        print("\n  Testing Sensor 1 (Channel 1):")
        pressure1 = controller.pressure_controller.read_pressure_value(1)
        print(f"    Pressure: {pressure1}")
        
        print("\n  TPG366 pressure controller tests completed successfully ✅")
        
    except Exception as e:
        logger.error(f"Error testing TPG366 pressure controller: {e}")
        print(f"  ❌ Error testing TPG366 pressure controller: {e}")
else:
    print("TPG366 pressure controller not connected")

Testing TPG366 Pressure Controller:
  Software Version: 010400
  Device Name: TPG366

  Testing Sensor 1 (Channel 1):
    Pressure: 1006.9999999999999

  TPG366 pressure controller tests completed successfully ✅


## 9. Monitoring Control Interface

In [30]:
# Create interactive controls for monitoring
start_button = widgets.Button(description="Start Monitoring", button_style='success')
stop_button = widgets.Button(description="Stop Monitoring", button_style='danger')
interval_slider = widgets.FloatSlider(
    value=5.0,
    min=0.1,
    max=5.0,
    step=0.1,
    description='Interval (s):',
    style={'description_width': 'initial'}
)
status_output = widgets.Output()

def on_start_clicked(b):
    with status_output:
        clear_output(wait=True)
        success = controller.start_monitoring(interval_slider.value)
        if success:
            print(f"✅ Monitoring started with {interval_slider.value}s interval")
        else:
            print("❌ Failed to start monitoring (already running?)")

def on_stop_clicked(b):
    with status_output:
        clear_output(wait=True)
        controller.stop_monitoring()
        print("🛑 Monitoring stopped")

start_button.on_click(on_start_clicked)
stop_button.on_click(on_stop_clicked)

# Display controls
controls = widgets.VBox([
    widgets.Label("Monitoring Controls:"),
    interval_slider,
    widgets.HBox([start_button, stop_button]),
    status_output
])

display(controls)

In [28]:
%matplotlib inline

## 10. Data Visualization

### 10.1 Static Plotting

In [17]:
def plot_monitoring_data():
    """Plot the collected monitoring data"""
    if not controller.monitoring_data:
        print("No monitoring data available. Start monitoring first.")
        return
    
    # Convert data to lists for easier plotting
    timestamps = []
    sensor1_pressure = []

    for cycle in controller.monitoring_data:
        timestamps.append(cycle['cycle_timestamp'])
        

        # Extract pressure data
        sensor1_pressure.append(cycle['sensor1_data']['pressure_value'] if cycle['sensor1_data'] and cycle['sensor1_data'].get('pressure_value') else None)

    # Create subplots
    fig, ax1 = plt.subplots(figsize=(16, 10))
    

    # Plot pressure (with logarithmic scale for vacuum)
    ax1.plot(timestamps, sensor1_pressure, 'b-', label='Sensor 1 Pressure', marker='^')
    ax1.set_yscale('log')
    ax1.set_ylabel('Pressure (log scale)')
    ax1.set_title('Pressure Over Time')
    ax1.legend()
    ax1.grid(True)
    

    # Rotate x-axis label for better readability
    ax1.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print(f"Plotted {len(controller.monitoring_data)} data points")
    valid_p1 = [p for p in sensor1_pressure if p is not None]

    if valid_p1:
        print(f"Sensor 1 pressure range: {min(valid_p1):.2e} to {max(valid_p1):.2e}")

def get_monitoring_summary():
    """Get a summary of current monitoring status"""
    print(f"Monitoring Status: {'🟢 Active' if controller.monitoring_active else '🔴 Inactive'}")
    print(f"Data Points Collected: {len(controller.monitoring_data)}")
    print(f"TPG366 Controller Connected: {'✅' if controller.pressure_controller else '❌'}")
    
    if controller.monitoring_data:
        latest = controller.monitoring_data[-1]
        print(f"\nLatest Data Point: {latest['cycle_timestamp']}")
        
        # Show latest pressure readings
        if latest['sensor1_data']:
            p1 = latest['sensor1_data'].get('pressure_value')
            print(f"  Sensor 1 Pressure: {p1}")

print("Data visualization functions ready")

Data visualization functions ready


In [ ]:
# Plot monitoring data (run this cell periodically to see updated plots)
plot_monitoring_data()

In [ ]:
# Get current monitoring summary
get_monitoring_summary()

### 10.2 Live Plotting (Same as original - no changes to logic)

In [31]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np

class PumpLockerPlotter:
    def __init__(self, controller, update_interval=1000):
        """
        Initialize live monitoring plot
        Args:
            controller: The controller object with monitoring_data
            update_interval: Update interval in milliseconds (default: 1000ms = 1 second)
        """
        self.controller = controller
        self.update_interval = update_interval

        # Initialize data lists
        self.timestamps = []
        self.sensor1_pressure = []

        # Create figure and subplots
        self.fig, self.ax2 = plt.subplots(figsize=(10, 6))

        self.temp_log = []
        self.log_file_size = 0

        # Animation object
        #self.animation = None

    def update_plot(self, frame):
        # Get new data
        current_size = len(self.controller.monitoring_data)
        if current_size > self.log_file_size:
            new_entries = self.controller.monitoring_data[self.log_file_size:]
            self.temp_log.extend(new_entries)
            self.log_file_size = current_size

        # Process new entries
        for entry in self.temp_log:
            self.timestamps.append(entry['cycle_timestamp'])
            
            # Extract pump data
            self.sensor1_pressure.append(entry['sensor1_data']['pressure_value'] if entry['sensor1_data'] else None)

        # Clear temp_log after processing to avoid reprocessing
        self.temp_log = []

        # Clear only the axes, not the entire figure
        self.ax2.clear()

        # Filter out None values for plotting
        valid_data2 = [(t, s1) for t, s1 in zip(self.timestamps, self.sensor1_pressure)
                      if s1 is not None]


        # Plot pressure data (ax2)
        if valid_data2:
            times2, sensor1_vals = zip(*valid_data2)

            # Plot sensor1 data (filter None values) - BLUE with circle marker (matches pump1)
            sensor1_valid = [(t, s) for t, s in zip(times2, sensor1_vals) if s is not None]
            if sensor1_valid:
                t1, s1 = zip(*sensor1_valid)
                if len(t1) > 120:
                    self.ax2.plot(t1[-119:], s1[-119:], 'b-', label='Sensor 1 Pressure', marker='o')
                else:
                    self.ax2.plot(t1, s1, 'b-', label='Sensor 1 Pressure', marker='o')


        # Use logarithmic scale for pressure if values are very small (typical for vacuum)
        self.ax2.set_yscale('log')
        self.ax2.set_ylabel('Pressure (log scale)')
        self.ax2.set_title('Pressure Over Time')
        self.ax2.legend()
        self.ax2.grid(True)


        # Rotate x-axis labels for better readability
        self.ax2.tick_params(axis='x', rotation=45)

        # Adjust layout
        self.fig.tight_layout()

    def start_live_plot(self):
        """Start the live plotting"""
        if not self.controller.monitoring_data:
            print("No monitoring data available. Start monitoring first.")
            return None, None

        print(f"Starting live plot with {self.update_interval/1000}s update interval...")
        print("Close the plot window to stop live plotting.")

        # Create animation
        self.ani = FuncAnimation(
            self.fig,
            self.update_plot,
            interval=self.update_interval,
            blit=False,  # Set to False for easier debugging
            cache_frame_data=False
        )

        # Show the plot
        plt.show()

        return self.timestamps, self.controller.monitoring_data

    def stop_live_plot(self):
        """Stop the live plotting"""
        if hasattr(self, 'ani'):
            self.ani.event_source.stop()
            print("Live plotting stopped.")
        else:
            print("No animation to stop.")

print("Live plotting class ready")

Live plotting class ready


In [32]:
%matplotlib notebook

In [33]:
# Start live plotting
live_plot = PumpLockerPlotter(controller)
timestamps, data = live_plot.start_live_plot()

<IPython.core.display.Javascript object>

Starting live plot with 1.0s update interval...
Close the plot window to stop live plotting.


In [34]:
# Stop live plotting
live_plot.stop_live_plot()

Live plotting stopped.


## 11. Data Export and Cleanup

In [35]:
def export_monitoring_data(filename: str = None):
    """Export monitoring data to CSV file"""
    if not controller.monitoring_data:
        print("No data to export")
        return
    
    if filename is None:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"pump_locker_data_{timestamp}.csv"
    
    # Flatten the data for CSV export
    flattened_data = []
    for cycle in controller.monitoring_data:
        row = {'timestamp': cycle['cycle_timestamp']}
        
        # Add pump 1 data
        if cycle['pump1_data']:
            for key, value in cycle['pump1_data'].items():
                row[f'pump1_{key}'] = value
        
        # Add pump 2 data
        if cycle['pump2_data']:
            for key, value in cycle['pump2_data'].items():
                row[f'pump2_{key}'] = value
        
        # Add sensor 1 data
        if cycle['sensor1_data']:
            for key, value in cycle['sensor1_data'].items():
                row[f'sensor1_{key}'] = value
        
        # Add sensor 2 data
        if cycle['sensor2_data']:
            for key, value in cycle['sensor2_data'].items():
                row[f'sensor2_{key}'] = value
        
        flattened_data.append(row)
    
    df = pd.DataFrame(flattened_data)
    df.to_csv(filename, index=False)
    logger.info(f"Data exported to {filename}")
    print(f"✅ Data exported to {filename}")

def save_monitoring_data_pickle(filename: str = None):
    """Save monitoring data as pickle file"""
    if not controller.monitoring_data:
        print("No data to save")
        return
        
    if filename is None:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"pump_locker_data_{timestamp}.pkl"
    
    with open(filename, 'wb') as f:
        pickle.dump(controller.monitoring_data, f)
    
    logger.info(f"Data saved to pickle file: {filename}")
    print(f"✅ Data saved to pickle file: {filename}")

def load_monitoring_data_pickle(filename: str):
    """Load monitoring data from pickle file"""
    try:
        with open(filename, 'rb') as f:
            loaded_data = pickle.load(f)
        
        controller.monitoring_data = loaded_data
        logger.info(f"Data loaded from pickle file: {filename}")
        print(f"✅ Data loaded from pickle file: {filename}")
        print(f"Loaded {len(loaded_data)} data points")
        
    except Exception as e:
        logger.error(f"Error loading data from {filename}: {e}")
        print(f"❌ Error loading data from {filename}: {e}")

print("Data export functions ready")

Data export functions ready


In [36]:
# Export current monitoring data
timestamp = datetime.now().strftime('%d-%m-%Y_%H%M')
export_monitoring_data(f"{timestamp}_pump_locker_data.csv")

2025-08-12 10:21:45 | INFO | Data exported to 12-08-2025_1021_pump_locker_data.csv


✅ Data exported to 12-08-2025_1021_pump_locker_data.csv


In [37]:
# Save monitoring data as pickle
timestamp = datetime.now().strftime('%d-%m-%Y_%H%M')
save_monitoring_data_pickle(f"{timestamp}_pump_locker_data.pkl")

2025-08-12 10:21:46 | INFO | Data saved to pickle file: 12-08-2025_1021_pump_locker_data.pkl


✅ Data saved to pickle file: 12-08-2025_1021_pump_locker_data.pkl


In [38]:
# Emergency disconnect all devices
controller.disconnect_all()

2025-08-12 10:21:48 | INFO | Disconnecting all devices...
2025-08-12 10:21:48 | INFO | Stopping continuous monitoring...
2025-08-12 10:21:49 | SUCCESS | Monitoring thread stopped successfully
2025-08-12 10:21:49 | INFO | Disconnected from Pfeiffer device pump_1
2025-08-12 10:21:49 | INFO | Pump 1 disconnected
2025-08-12 10:21:50 | INFO | Disconnected from Pfeiffer device pump_2
2025-08-12 10:21:50 | INFO | Pump 2 disconnected
2025-08-12 10:21:50 | INFO | Disconnected from Pfeiffer device tpg366_pressure
2025-08-12 10:21:50 | INFO | TPG366 pressure controller disconnected
2025-08-12 10:21:50 | SUCCESS | All devices disconnected


In [35]:
# Emergency stop and cleanup
print("🛑 Emergency Stop - Disconnecting all devices...")
controller.disconnect_all()
print("✅ All devices disconnected safely")
print("\nTo reconnect, run the connection cells again.")

2025-09-09 15:35:31 | INFO | Disconnecting all devices...
2025-09-09 15:35:31 | INFO | Monitoring is not currently active
2025-09-09 15:35:31 | INFO | Disconnected from Pfeiffer device tpg366_pressure
2025-09-09 15:35:31 | INFO | TPG366 pressure controller disconnected
2025-09-09 15:35:31 | SUCCESS | All devices disconnected


🛑 Emergency Stop - Disconnecting all devices...
✅ All devices disconnected safely

To reconnect, run the connection cells again.


## Quick Reference

### Key Functions:
- **Connect devices**: Run cells in section 6
- **Start monitoring**: Use the interface in section 9 or call `controller.start_monitoring(interval)`
- **Stop monitoring**: Use the interface in section 9 or call `controller.stop_monitoring()`
- **Control pumps**: Use functions in section 8 (enable/disable individual or both pumps)
- **View data**: Run `get_monitoring_summary()` and `plot_monitoring_data()`
- **Export data**: Run `export_monitoring_data()` or `save_monitoring_data_pickle()`
- **Emergency stop**: Run the last cell to disconnect everything

### Key Improvements from Original:
- Uses current hardware APIs (HiScroll12 and TPG366 classes)
- Shared logging system with loguru for all devices
- Simplified device initialization with shared logger and thread lock
- Updated monitoring functions to use current device methods
- Live plotting logic kept unchanged as requested
- Proper error handling and status reporting
- Support for both CSV and pickle data export

### Device Configuration:
- Pump 1 & 2: HiScroll12 pumps (standard address 2)
- Pressure Controller: TPG366 (standard address 1, channels 1 & 2)
- All devices use shared logger and thread synchronization